# PART2--->DB--->1 -->IMPLEMENTORY MANAGEMENT IN PYTHON WITH MYSQL
PROGRAMMING WITH MYSQL
#INVENTORY MANAGEMENT
import mysql.connector

# Establish a connection to the MySQL database(INVENTORY)
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="kamali213",
    database="inventory_db"
)

cursor = conn.cursor()

def add_product(name, description, price, quantity):
    query = "INSERT INTO products (name, description, price, quantity) VALUES (%s, %s, %s, %s)"
    data = (name, description, price, quantity)
    cursor.execute(query, data)
    conn.commit()
    print("Product added successfully!")

def update_product(product_id, quantity_change):
    query = "UPDATE products SET quantity = quantity + %s WHERE id = %s"
    data = (quantity_change, product_id)
    cursor.execute(query, data)
    conn.commit()
    print("Inventory updated successfully!")

def view_products():
    query = "SELECT * FROM products"
    cursor.execute(query)
    products = cursor.fetchall()
    if not products:
        print("No products found.")
    else:
        for product in products:
            print(product)

def delete_product(product_id):
    query = "DELETE FROM products WHERE id = %s"
    data = (product_id,)
    cursor.execute(query, data)
    conn.commit()
    print("Product deleted successfully!")

while True:
    print("\nInventory Management System")
    print("1. Add Product")
    print("2. Update Inventory")
    print("3. View Products")
    print("4. Delete Product")
    print("5. Quit")

    choice = input("Enter your choice: ")

    if choice == "1":
        name = input("Enter product name: ")
        description = input("Enter product description: ")
        price = float(input("Enter product price: "))
        quantity = int(input("Enter product quantity: "))
        add_product(name, description, price, quantity)
    elif choice == "2":
        product_id = int(input("Enter product ID to update inventory: "))
        quantity_change = int(input("Enter quantity change: "))
        update_product(product_id, quantity_change)
    elif choice == "3":
        view_products()
    elif choice == "4":
        product_id = int(input("Enter product ID to delete: "))
        delete_product(product_id)
    elif choice == "5":
        break
    else:
        print("Invalid choice. Please try again.")

# Close the cursor and database connection
cursor.close()
conn.close()


# PART2---->db----->2)CUSTOMER ORDER PROCESSING
customer order processing Scenario: you are building a customer order processing system for an e-commence company. The system needs to interact with a MYSQL database to store customer order, products, and other details. Design a MYSQL database schema for the order processing system, including tables for customers, products and orders. Write a python program that connects to the database and allows customers to place new orders. Implement a feature that calculate that total cost of an order and updates product quantities in the database. How would you handle cases where a product is no longer available when a customer places an order ? .Develop a function to generate order for the company's finance department.

In [8]:
import mysql.connector
from datetime import date

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="kamali213",
    database="order_processing_db"
)

cursor = conn.cursor()

def place_order(customer_id, product_id, quantity):
    try:
        # Check if the product is available in sufficient quantity
        cursor.execute("SELECT price, quantity_available FROM products WHERE product_id = %s", (product_id,))
        result = cursor.fetchone()

        if result is None:
            print("Product not found.")
            return
        elif result[1] < quantity:
            print("Insufficient quantity available.")
            return

        # Calculate the subtotal
        subtotal = result[0] * quantity

        # Insert into orders table
        cursor.execute("INSERT INTO orders (customer_id, order_date, total_cost) VALUES (%s, %s, %s)",
                       (customer_id, date.today(), subtotal))
        order_id = cursor.lastrowid

        # Insert into order_details table
        cursor.execute("INSERT INTO order_details (order_id, product_id, quantity_ordered, subtotal) VALUES (%s, %s, %s, %s)",
                       (order_id, product_id, quantity, subtotal))

        # Update product quantity
        cursor.execute("UPDATE products SET quantity_available = quantity_available - %s WHERE product_id = %s",
                       (quantity, product_id))

        conn.commit()
        print("Order placed successfully!")

    except Exception as e:
        conn.rollback()
        print(f"Error placing order: {e}")

# Example usage:
customer_id = 1  # Replace with the actual customer ID
product_id = 1   # Replace with the actual product ID
quantity = 3

place_order(customer_id, product_id, quantity)

# Close the cursor and database connection
cursor.close()
conn.close()


Product not found.


In [9]:
import mysql.connector
from datetime import date

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="kamali213",
    database="order_processing_db"
)

cursor = conn.cursor()

def calculate_total_cost(order_id):
    try:
        # Calculate the total cost of an order
        cursor.execute("SELECT SUM(subtotal) FROM order_details WHERE order_id = %s", (order_id,))
        total_cost = cursor.fetchone()[0] or 0.0
        return total_cost

    except Exception as e:
        print(f"Error calculating total cost: {e}")
        return None

def place_order(customer_id, product_id, quantity):
    try:
        # Check if the product is available in sufficient quantity
        cursor.execute("SELECT price, quantity_available FROM products WHERE product_id = %s", (product_id,))
        result = cursor.fetchone()

        if result is None:
            print("Product not found.")
            return
        elif result[1] < quantity:
            print("Insufficient quantity available.")
            return

        # Calculate the subtotal
        subtotal = result[0] * quantity

        # Insert into orders table
        cursor.execute("INSERT INTO orders (customer_id, order_date, total_cost) VALUES (%s, %s, %s)",
                       (customer_id, date.today(), 0.0))
        order_id = cursor.lastrowid

        # Insert into order_details table
        cursor.execute("INSERT INTO order_details (order_id, product_id, quantity_ordered, subtotal) VALUES (%s, %s, %s, %s)",
                       (order_id, product_id, quantity, subtotal))

        # Update product quantity
        cursor.execute("UPDATE products SET quantity_available = quantity_available - %s WHERE product_id = %s",
                       (quantity, product_id))

        # Calculate the total cost and update the order
        total_cost = calculate_total_cost(order_id)
        cursor.execute("UPDATE orders SET total_cost = %s WHERE order_id = %s", (total_cost, order_id))

        conn.commit()
        print("Order placed successfully!")

    except Exception as e:
        conn.rollback()
        print(f"Error placing order: {e}")

# Example usage:
customer_id = 1  # Replace with the actual customer ID
product_id = 1   # Replace with the actual product ID
quantity = 3

place_order(customer_id, product_id, quantity)

# Close the cursor and database connection
cursor.close()
conn.close()


Product not found.


In [ ]:
def place_order(customer_id, product_id, quantity):
    try:
        # Check if the product is available in sufficient quantity
        cursor.execute("SELECT price, quantity_available FROM products WHERE product_id = %s", (product_id,))
        result = cursor.fetchone()

        if result is None:
            print("Product not found.")
            return
        elif result[1] < quantity:
            print("Insufficient quantity available. Order cannot be placed.")
            return

        # Calculate the subtotal
        subtotal = result[0] * quantity

        # Start a database transaction
        conn.start_transaction()

        # Insert into orders table
        cursor.execute("INSERT INTO orders (customer_id, order_date, total_cost) VALUES (%s, %s, %s)",
                       (customer_id, date.today(), 0.0))
        order_id = cursor.lastrowid

        # Insert into order_details table
        cursor.execute("INSERT INTO order_details (order_id, product_id, quantity_ordered, subtotal) VALUES (%s, %s, %s, %s)",
                       (order_id, product_id, quantity, subtotal))

        # Update product quantity
        cursor.execute("UPDATE products SET quantity_available = quantity_available - %s WHERE product_id = %s",
                       (quantity, product_id))

        # Commit the transaction
        conn.commit()

        # Calculate the total cost and update the order
        total_cost = calculate_total_cost(order_id)
        cursor.execute("UPDATE orders SET total_cost = %s WHERE order_id = %s", (total_cost, order_id))

        conn.commit()
        print("Order placed successfully!")

    except Exception as e:
        conn.rollback()  # Rollback the transaction in case of an error
        print(f"Error placing order: {e}")

# Example usage:
customer_id = 1  # Replace with the actual customer ID
product_id = 1   # Replace with the actual product ID
quantity = 3

place_order(customer_id, product_id, quantity)


In [11]:
import mysql.connector

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="kamali213",
    database="order_processing_db"
)

cursor = conn.cursor()

def generate_order_for_finance(order_id):
    try:
        # Fetch order details
        cursor.execute("SELECT customers.first_name, customers.last_name, orders.order_date, orders.total_cost "
                       "FROM orders "
                       "INNER JOIN customers ON orders.customer_id = customers.customer_id "
                       "WHERE orders.order_id = %s", (order_id,))
        order_data = cursor.fetchone()

        if order_data is None:
            print("Order not found.")
            return

        first_name, last_name, order_date, total_cost = order_data

        # Format the order for the finance department
        order_info = f"Order ID: {order_id}\n"
        order_info += f"Customer Name: {first_name} {last_name}\n"
        order_info += f"Order Date: {order_date}\n"
        order_info += f"Total Cost: ${total_cost:.2f}"

        # You can customize further formatting or save this information to a file or send it as an email to the finance department.

        print("Finance Department Order Information:")
        print(order_info)

    except Exception as e:
        print(f"Error generating order for finance: {e}")

# Example usage:
order_id = 1  # Replace with the actual order ID
generate_order_for_finance(order_id)

# Close the cursor and database connection
cursor.close()
conn.close()


Order not found.


In [14]:
import mysql.connector

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="kamali213",
    database="order_processing_db"
)

cursor = conn.cursor()

def generate_order_for_finance(order_id):
    try:
        # Fetch order details
        cursor.execute("SELECT customers.first_name, customers.last_name, orders.order_date, orders.total_cost "
                       "FROM orders "
                       "INNER JOIN customers ON orders.customer_id = customers.customer_id "
                       "WHERE orders.order_id = %s", (order_id,))
        order_data = cursor.fetchone()

        if order_data is None:
            print("Order not found.")
            return

        first_name, last_name, order_date, total_cost = order_data

        # Format the order for the finance department
        order_info = f"Order ID: {order_id}\n"
        order_info += f"Customer Name: {first_name} {last_name}\n"
        order_info += f"Order Date: {order_date}\n"
        order_info += f"Total Cost: ${total_cost:.2f}"

        # You can customize further formatting or save this information to a file or send it as an email to the finance department.

        print("Finance Department Order Information:")
        print(order_info)

    except Exception as e:
        print(f"Error generating order for finance: {e}")

# Example usage:
order_id = 1  # Replace with the actual order ID
generate_order_for_finance(order_id)

# Close the cursor and database connection
cursor.close()
conn.close()


Order not found.


# PART2----->DB---->4---> you are develpoing a employee management

In [19]:
import mysql.connector

# Database connection parameters
db_config = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "kamali213",
    "database": "employee_db"
}

# Department ID for the specific department you want to retrieve employees from
department_id = 1  # Replace with the actual department ID

try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(**db_config)

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Query to retrieve employees in a specific department
    query = """
        SELECT employee_id, first_name, last_name, email
        FROM employees
        WHERE department_id = %s
    """

    # Execute the query with the department_id as a parameter
    cursor.execute(query, (department_id,))

    # Fetch all rows (employees) from the result set
    employees = cursor.fetchall()

    # Print the list of employees in the specific department
    if employees:
        print("Employees in Department:")
        for employee in employees:
            employee_id, first_name, last_name, email = employee
            print(f"Employee ID: {employee_id}, Name: {first_name} {last_name}, Email: {email}")
    else:
        print("No employees found in the specified department.")

except mysql.connector.Error as e:
    print(f"Error connecting to the database: {e}")

finally:
    # Close the cursor and database connection
    if "cursor" in locals() and cursor is not None:
        cursor.close()
    if "conn" in locals() and conn.is_connected():
        conn.close()


Error connecting to the database: 1146 (42S02): Table 'employee_db.employees' doesn't exist


# 2) virtual environment management. you are working on multiple python projects with the different dependencies and versions.to avoid and ensure project isolation, you decide to use virtual environments. create a python program that demonstrates for a specific projects. create a virtual environment for a specific project. activate and deactivate virtual environments. install upgrade and uninstall packages within a virtual environments. List the installed packages in virtual environment. implement error handling for virtual environment operation

In [1]:
import subprocess
import sys
import os

def create_virtual_environment(project_name):
    try:
        # Create a virtual environment for the specified project
        subprocess.run([sys.executable, '-m', 'venv', project_name])
        print(f"Virtual environment '{project_name}' created successfully.")
    except Exception as e:
        print(f"Error creating virtual environment: {e}")

def activate_virtual_environment(venv_path):
    try:
        # Activate the virtual environment
        if sys.platform == 'win32':
            activate_script = os.path.join(venv_path, 'Scripts', 'activate')
        else:
            activate_script = os.path.join(venv_path, 'bin', 'activate')

        activate_cmd = f'source {activate_script}'
        subprocess.run(activate_cmd, shell=True)
        print(f"Activated virtual environment at '{venv_path}'.")
    except Exception as e:
        print(f"Error activating virtual environment: {e}")

def deactivate_virtual_environment():
    try:
        # Deactivate the virtual environment
        subprocess.run(['deactivate'], shell=True)
        print("Deactivated virtual environment.")
    except Exception as e:
        print(f"Error deactivating virtual environment: {e}")

def install_package(venv_path, package_name):
    try:
        # Install a package in the virtual environment
        subprocess.run([os.path.join(venv_path, 'bin', 'pip'), 'install', package_name])
        print(f"Package '{package_name}' installed in the virtual environment.")
    except Exception as e:
        print(f"Error installing package: {e}")

def list_installed_packages(venv_path):
    try:
        # List installed packages in the virtual environment
        subprocess.run([os.path.join(venv_path, 'bin', 'pip'), 'freeze'])
    except Exception as e:
        print(f"Error listing installed packages: {e}")

def main():
    project_name = 'my_project'
    create_virtual_environment(project_name)

    # Specify the virtual environment path
    venv_path = os.path.join(os.getcwd(), project_name)

    activate_virtual_environment(venv_path)

    # Install or upgrade packages within the virtual environment
    install_package(venv_path, 'requests')
    install_package(venv_path, '--upgrade pandas')

    # List installed packages
    list_installed_packages(venv_path)

    # Deactivate the virtual environment
    deactivate_virtual_environment()

if __name__ == "__main__":
    main()


Virtual environment 'my_project' created successfully.
Activated virtual environment at 'C:\Users\hp\Documents\my_project'.
Error installing package: [WinError 2] The system cannot find the file specified
Error installing package: [WinError 2] The system cannot find the file specified
Error listing installed packages: [WinError 2] The system cannot find the file specified
Deactivated virtual environment.


# 3  Module dependency solution. You are developing a python application that relies on the third party packages. managing dependency and ensuring compatibility is crucial for your projects success. design a python program that demonstrates the following. use a requirements text file to specify project dependencies. Automatically install all project dependencies from the requirements text file. Ensure that the versions of installed packages are compatible. Implement error handling for dependency resolution and installation.

In [ ]:
import subprocess

def install_dependencies(requirements_file):
    try:
        # Execute the 'pip install' command to install dependencies
        subprocess.run(['pip', 'install', '-r', requirements_file], check=True)
        print("Dependencies installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error during dependency installation (Exit Code {e.returncode}):")
        print(e.stderr.decode())
    except FileNotFoundError:
        print("Error: 'pip' command not found. Please make sure pip is installed.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def main():
    # Specify the path to your requirements.txt file
    requirements_file = 'requirements.txt'

    # Call the function to install dependencies
    install_dependencies(requirements_file)

if __name__ == "__main__":
    main()


# PART2----->DB--->3 )You are tasked with developing a Python program that connects to a MySQL database, retriev data from a table, performs some operations on the data, and updates the database with th modified data. Please write Python code to accomplish this task.

Instructions:

1. Assume that you have a MySQL database server running with the following details:

LHost: localhost

1. Port: 3306

I. Username: your username Iv. Password: your password

V. Database Name your database

vi. Table Name: your table

vi. The table has the following columns id (int), name (varchar),

quantity (int). 2. Your Python program should

Connect to the MySQL database

Retrieve all records from the your table table. Calculate the total quantity of all records retrieved

Update the quantity column of each record by doubling its value Commit the changes to the database.

vi. Close the database connection.

3. Handle any potential errors that may occur during the database connection and data manipulation, such as connection failures or SQL errors. Provide comments in your code to explain each step 

In [7]:
import mysql.connector

# Database connection parameters
db_config = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "kamali213",
    "database": "order_processing_db"
}

try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(**db_config)

    if conn.is_connected():
        print("Connected to the database")

        # Create a cursor object to interact with the database
        cursor = conn.cursor()

        # Retrieve all records from the 'your_table' table
        cursor.execute("SELECT product_id, product_name, quantity_available  FROM products")
        records = cursor.fetchall()

        # Calculate the total quantity
        total_quantity = sum(record[2] for record in records)

        # Double the quantity of each record and update the database
        for record in records:
            new_quantity = record[2] * 2
            cursor.execute("UPDATE your_table SET quantity = %s WHERE id = %s", (new_quantity, record[0]))

        # Commit the changes to the database
        conn.commit()
        print("Changes committed successfully.")

except mysql.connector.Error as e:
    print(f"Error connecting to the database: {e}")

except mysql.connector.DatabaseError as e:
    print(f"Database error: {e}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    # Close the cursor and database connection
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()


Connected to the database
Changes committed successfully.


# PART2---->DB--->4 You are developing an employee management system for a company. The database should store employee information, Including name, salary, department, and hire date. Managers she be able to view and update employee details.

1. Design the database schema for the employee management system
Write Python code to connect to the database and retrieve a list of employees in a specific department.
.Implement a feature to update an employee's salary.

In [10]:
import mysql.connector

def get_employees_in_department(department_name):
    try:
        # Database connection parameters
        db_config = {
            "host": "127.0.0.1",
            "port": 3306,
            "user": "root",
            "password": "kamali213",
            "database": "employee_db"
        }

        # Establish a connection to the MySQL database
        conn = mysql.connector.connect(**db_config)
        
        if conn.is_connected():
            # Create a cursor object to interact with the database
            cursor = conn.cursor()

            # Retrieve employees in a specific department
            query = """
                SELECT employee_id, first_name, last_name
                FROM Employees
                WHERE department_id = (
                    SELECT department_id
                    FROM Departments
                    WHERE department_name = %s
                )
            """
            cursor.execute(query, (department_name,))

            # Fetch all records
            employees = cursor.fetchall()

            if employees:
                print(f"Employees in the '{department_name}' department:")
                for employee in employees:
                    print(f"Employee ID: {employee[0]}, Name: {employee[1]} {employee[2]}")
            else:
                print(f"No employees found in the '{department_name}' department.")

    except mysql.connector.Error as e:
        print(f"Error connecting to the database: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    finally:
        # Close the cursor and database connection
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if conn.is_connected():
            conn.close()

# Specify the department name
department_name = "Sales"

# Call the function to retrieve employees in the specified department
get_employees_in_department(department_name)


No employees found in the 'Sales' department.
